# query optimization

the process of query optimization is very similar to the process of refactoring.

the goal is to make the query more efficient without changing it's behavior.

so ideally you should write TDD-style tests for your queries first before you start optimizing them to avoid any unexpected behavior changes.

in this notebook we will go through some common query optimization techniques based on examples.

In [ ]:
!chmod +x ./reset.sh && ./reset.sh > /dev/null
!psql postgres -c "reset all;"

In [70]:
import os
import sys
import json

def equals_query(query1: str, query2: str, prefix: str="") -> bool:
    output1 = os.popen(f'psql postgres -c "{prefix} {query1};"').read()
    output2 = os.popen(f'psql postgres -c "{prefix} {query2};"').read()
    length_match = len(output1) == len(output2)
    
    output1 = output1.splitlines()
    output2 = output2.splitlines()
    output1.sort()
    output2.sort()
    output1 = "\n".join(output1)
    output2 = "\n".join(output2)
    non_order_match = output1 == output2
    
    return length_match and non_order_match

def print_simple_execution_plan(query: str, prefix: str="") -> None:
    output = os.popen(f'psql postgres -c "{prefix} explain (analyze, verbose off, costs off, settings off, generic_plan off, buffers off, wal off, timing off, summary off) {query};"').read()
    print(output)

def print_verbose_execution_plan(query: str, prefix: str="") -> None:
    output = os.popen(f'psql postgres -c "{prefix} explain (analyze, verbose, costs, settings, buffers, wal, timing, summary) {query};"').read()
    print(output)

def print_avg_time(query: str, prefix: str="", iters:int=10) -> None:   
    def get_exec_time(query: str, prefix: str="") -> float:
        output = os.popen(f'psql postgres -c "{prefix} explain (analyze, verbose, costs, settings, buffers, wal, timing, summary, format json) {query};"').read()
        output = output.splitlines()[2:-2]
        for i in range(len(output)):
            if len(output[i]) > 2:
                output[i] = output[i][:-1]
        output = "\n".join(output)
        output = json.loads(output)[0]
        return float(output["Execution Time"])

    vals = []
    for _ in range(iters):
        vals.append(get_exec_time(query, prefix))
        sys.stdout.write(f"\r{iters - _} iterations left")
        sys.stdout.write("\r")
        sys.stdout.flush()
    print(f"average execution time in {iters} interations: {sum(vals) / iters:.2f} ms")

# demo
demo_q1 = "SELECT a,b,c FROM r NATURAL JOIN s NATURAL JOIN t"
demo_q2 = "SELECT a,b,c FROM r NATURAL JOIN s NATURAL JOIN t ORDER BY a,b,c"
assert equals_query(demo_q1, demo_q2)
# print_simple_execution_plan(demo_q1)
# print_verbose_execution_plan(demo_q1)
print_avg_time(demo_q1)


average execution time in 10 interations: 325.58 ms


# example a) text search query

original query: `76.858 ms`

- uses substring to search for a specific word in the text column
- to learn more about postgres substring function and string matching:
    - https://www.postgresql.org/docs/9.1/functions-string.html
    - https://www.postgresql.org/docs/9.1/functions-matching.html

```sql
SELECT COUNT(*) FROM comments
WHERE substring(text from '%#" hello #"%' for '#') IS NOT NULL
OR substring(text from '%#" hi #"%' for '#') IS NOT NULL;
```

---

better standard library: `48.933 ms`

- the substring function is not meant for text matching

```sql
SELECT COUNT(*) FROM comments WHERE text LIKE '% hello %' OR text LIKE '% hi %';
```

---

index on search field: `31.11 ms`

- improving lookup time by adding an index on the text column

```sql
CREATE INDEX text_index ON comments (text);
SELECT COUNT(*) FROM comments WHERE text LIKE '% hello %' OR text LIKE '% hi %';
```

---

precompute the results: `0.3 ms`

- we can add a new column to the table to store the result of the search that we have to set on each insert or update
- add an index on the new column to improve lookup time
- can be done with a trigger but it could be considered cheating because it's less about query optimization and more about data optimization
- there are much nicer ways to do this, as we will see in the next example

```sql
ALTER TABLE comments ADD COLUMN is_match boolean;
UPDATE comments SET is_match = TRUE WHERE text LIKE '% hello %' OR text LIKE '% hi %';
CREATE INDEX is_match_index ON comments (is_match);
SELECT count(*) FROM comments WHERE is_match = TRUE;
```

### simplify the string matching

In [326]:
# equivalent
!psql postgres -c "select substring('this is swaggy' from '%#\" is #\"%' for '#');"
!psql postgres -c "select substring('this is swaggy' from '%@\" is @\"%' for '@');"
!psql postgres -c "select substring('this is swaggy' from '%$\" is $\"%' for '$');"
!psql postgres -c "select substring('this is swaggy', ' is ');"

# simplified parts of original query
!psql postgres -c "SELECT * FROM comments WHERE substring(text from '%#\" hi #\"%' for '#') IS NOT NULL;"
!psql postgres -c "SELECT * FROM comments WHERE substring(text, ' hi ') IS NOT NULL;"

 substring 
-----------
  is 
(1 row)

 substring 
-----------
  is 
(1 row)

 substring 
-----------
  is 
(1 row)

 substring 
-----------
  is 
(1 row)

  id   | postid | score |                                                                                                                                           text                                                                                                                                            |      creationdate       | userid | userdisplayname | contentlicense 
-------+--------+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------+--------+-----------------+----------------
 37171 |  17355 |     1 | the infrared radiation from the sun is shortwave infrared an

In [339]:
# original
!psql postgres -c "SELECT * FROM comments WHERE substring(text from '%#\" hello #\"%' for '#') IS NOT NULL OR substring(text from '%#\" hi #\"%' for '#') IS NOT NULL;"

# simplified
!psql postgres -c "SELECT * FROM comments WHERE substring(text, ' hello ') IS NOT NULL OR substring(text, ' hi ') IS NOT NULL;"
!psql postgres -c "SELECT * FROM comments WHERE text LIKE '% hello %' OR text LIKE '% hi %';"

  id   | postid | score |                                                                                                                                           text                                                                                                                                            |      creationdate       | userid | userdisplayname | contentlicense 
-------+--------+-------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------+--------+-----------------+----------------
 37171 |  17355 |     1 | the infrared radiation from the sun is shortwave infrared and is not stopped by the atmosphere,the earth emits infrared radiation in longwave radiation and a lot of this gets absorbed and radiated back to the suface 

In [343]:
# checking performance (we can't use my library because it breaks on nested double quotes)

!psql postgres -c "explain (analyze) SELECT count(*) FROM comments WHERE substring(text, ' hello ') IS NOT NULL OR substring(text, ' hi ') IS NOT NULL;"
!psql postgres -c "explain (analyze) SELECT count(*) FROM comments WHERE text LIKE '% hello %' OR text LIKE '% hi %';"

                                                     QUERY PLAN                                                      
---------------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=1475.85..1475.86 rows=1 width=8) (actual time=76.807..76.807 rows=1 loops=1)
   ->  Seq Scan on comments  (cost=0.00..1412.44 rows=25362 width=0) (actual time=61.066..76.797 rows=1 loops=1)
         Filter: (("substring"(text, ' hello '::text) IS NOT NULL) OR ("substring"(text, ' hi '::text) IS NOT NULL))
         Rows Removed by Filter: 25362
 Planning Time: 0.274 ms
 Execution Time: 76.858 ms
(6 rows)

                                                 QUERY PLAN                                                  
-------------------------------------------------------------------------------------------------------------
 Aggregate  (cost=1412.46..1412.47 rows=1 width=8) (actual time=48.866..48.867 rows=1 loops=1)
   ->  Seq Scan on comm

### optimize through trial and error

In [368]:
# create index on text
!psql postgres -c "CREATE INDEX text_index ON comments (text);"
print_avg_time("SELECT count(*) FROM comments WHERE text LIKE '% hello %' OR text LIKE '% hi %';")

ERROR:  relation "text_index" already exists
average execution time in 10 interations: 31.11 ms


In [374]:
# new smaller table with index (actually slower lol)
!psql postgres -c "CREATE TABLE comments_small AS SELECT text FROM comments;"
!psql postgres -c "CREATE INDEX text_index_small ON comments_small (text);"
print_avg_time("SELECT COUNT(*) FROM comments WHERE text LIKE '% hello %' OR text LIKE '% hi %';")

ERROR:  relation "comments_small" already exists
ERROR:  relation "text_index_small" already exists
average execution time in 10 interations: 32.58 ms


In [378]:
# union all
!psql postgres -c "SELECT * FROM (SELECT 1 FROM comments WHERE text LIKE '% hello %' UNION ALL SELECT 1 FROM comments WHERE text LIKE '% hi %') AS temp;"
print_avg_time("SELECT count(*) FROM (SELECT 1 FROM comments WHERE text LIKE '% hello %' UNION ALL SELECT 1 FROM comments WHERE text LIKE '% hi %') AS temp;")

 ?column? 
----------
        1
(1 row)

average execution time in 10 interations: 34.73 ms


In [381]:
# add "is_match" column to table (can be generated with a trigger)
!psql postgres -c "ALTER TABLE comments ADD COLUMN is_match boolean;"
!psql postgres -c "UPDATE comments SET is_match = TRUE WHERE text LIKE '% hello %' OR text LIKE '% hi %';"
# add index to is_match
!psql postgres -c "CREATE INDEX is_match_index ON comments (is_match);"
print_avg_time("SELECT count(*) FROM comments WHERE is_match = TRUE;")

ERROR:  column "is_match" of relation "comments" already exists
UPDATE 1
ERROR:  relation "is_match_index" already exists
average execution time in 10 interations: 0.11 ms


In [382]:
print_simple_execution_plan("SELECT count(*) FROM comments WHERE text LIKE '% hello %' OR text LIKE '% hi %';")

                                QUERY PLAN                                 
---------------------------------------------------------------------------
 Aggregate (actual rows=1 loops=1)
   ->  Seq Scan on comments (actual rows=1 loops=1)
         Filter: ((text ~~ '% hello %'::text) OR (text ~~ '% hi %'::text))
         Rows Removed by Filter: 25362
(4 rows)




# example b) nested query

original query: `44819.041 ms`

```sql
SELECT DISTINCT b.name FROM badges b
WHERE b.userid IN (
    SELECT u.id from users u WHERE u.reputation < 0.5 * b.class
);
```

---

use natural join instead of nested query: `11.07 ms`

```sql
SELECT DISTINCT b.name FROM badges b JOIN users u ON b.userid = u.id WHERE u.reputation < 0.5 * b.class;
```

---

### test inital query

In [386]:
!psql postgres -c "EXPLAIN ANALYSE SELECT DISTINCT b.name FROM badges b WHERE b.userid IN (SELECT u.id from users u WHERE u.reputation < 0.5 * b.class);"

                                                           QUERY PLAN                                                            
---------------------------------------------------------------------------------------------------------------------------------
 Unique  (cost=9651385.45..9651448.70 rows=86 width=10) (actual time=44818.861..44818.964 rows=29 loops=1)
   ->  Sort  (cost=9651385.45..9651417.07 rows=12650 width=10) (actual time=44818.860..44818.904 rows=2408 loops=1)
         Sort Key: b.name
         Sort Method: quicksort  Memory: 149kB
         ->  Seq Scan on badges b  (cost=0.00..9650523.55 rows=12650 width=10) (actual time=92.086..44818.214 rows=2408 loops=1)
               Filter: (SubPlan 1)
               Rows Removed by Filter: 22891
               SubPlan 1
                 ->  Seq Scan on users u  (cost=0.00..750.88 rows=4798 width=4) (actual time=0.197..1.646 rows=5707 loops=25299)
                       Filter: ((reputation)::numeric < (0.5 * (b.class)::numeric

In [383]:
print_simple_execution_plan("SELECT DISTINCT b.name FROM badges b WHERE b.userid IN (SELECT u.id from users u WHERE u.reputation < 0.5 * b.class);")

                                     QUERY PLAN                                     
------------------------------------------------------------------------------------
 Unique (actual rows=29 loops=1)
   ->  Sort (actual rows=2408 loops=1)
         Sort Key: b.name
         Sort Method: quicksort  Memory: 149kB
         ->  Seq Scan on badges b (actual rows=2408 loops=1)
               Filter: (SubPlan 1)
               Rows Removed by Filter: 22891
               SubPlan 1
                 ->  Seq Scan on users u (actual rows=5707 loops=25299)
                       Filter: ((reputation)::numeric < (0.5 * (b.class)::numeric))
                       Rows Removed by Filter: 8062
(11 rows)




### remove nested query with natural join

In [389]:
q1 = "SELECT DISTINCT b.name FROM badges b WHERE b.userid IN (SELECT u.id from users u WHERE u.reputation < 0.5 * b.class);"
q2 = "SELECT DISTINCT b.name FROM badges b JOIN users u ON b.userid = u.id WHERE u.reputation < 0.5 * b.class;"
print(equals_query(q1, q2))

True


In [408]:
print_avg_time("SELECT DISTINCT b.name FROM badges b JOIN users u ON b.userid = u.id WHERE u.reputation < 0.5 * b.class;")

average execution time in 10 interations: 11.07 ms


In [405]:
print_simple_execution_plan("SELECT DISTINCT b.name FROM badges b JOIN users u ON b.userid = u.id WHERE u.reputation < 0.5 * b.class;")

                                 QUERY PLAN                                  
-----------------------------------------------------------------------------
 HashAggregate (actual rows=29 loops=1)
   Group Key: b.name
   Batches: 1  Memory Usage: 40kB
   ->  Hash Join (actual rows=2408 loops=1)
         Hash Cond: (b.userid = u.id)
         Join Filter: ((u.reputation)::numeric < (0.5 * (b.class)::numeric))
         Rows Removed by Join Filter: 22891
         ->  Seq Scan on badges b (actual rows=25299 loops=1)
         ->  Hash (actual rows=14394 loops=1)
               Buckets: 16384  Batches: 1  Memory Usage: 691kB
               ->  Seq Scan on users u (actual rows=14394 loops=1)
(11 rows)




### optimize through trial and error

In [409]:
# too many indexes actually make it slower
!psql postgres -c "CREATE INDEX badges_user_id ON badges (userid);"
# !psql postgres -c "CREATE INDEX badges_class ON badges (class);"
!psql postgres -c "CREATE INDEX users_id ON users (id);"
# !psql postgres -c "CREATE INDEX users_reputation ON users (reputation);"
print_avg_time("SELECT DISTINCT b.name FROM badges b JOIN users u ON b.userid = u.id WHERE u.reputation < 0.5 * b.class;")

CREATE INDEX
CREATE INDEX
average execution time in 10 interations: 11.40 ms


In [404]:
# use materialized view
!psql postgres -c "CREATE MATERIALIZED VIEW user_badge AS SELECT b.userid, b.name, u.reputation, b.class FROM badges b JOIN users u ON b.userid = u.id;"
print_avg_time("SELECT DISTINCT name FROM user_badge WHERE reputation < 0.5 * class;")
print(equals_query("SELECT DISTINCT b.name FROM badges b JOIN users u ON b.userid = u.id WHERE u.reputation < 0.5 * b.class;", "SELECT DISTINCT name FROM user_badge WHERE reputation < 0.5 * class;"))

# use index
!psql postgres -c "CREATE INDEX reputation_index ON users (reputation);"
!psql postgres -c "CREATE INDEX class_index ON badges (class);"

SELECT 25299
average execution time in 10 interations: 8.36 ms
True
CREATE INDEX
CREATE INDEX
